### Direct Open Loop Controller Analysis

This notebook contains the analysis of the direct open loop controller across temperatures.

In [3]:
import import_ipynb
from kinetic_parameters import rate

In [2]:
rate(controller="direct_open_loop", temperature_C=25)

{'alpha_X': 0.1,
 'alpha_Y': 0.001,
 'alpha_Y+': 1.0,
 'delta_X': 0.0005,
 'delta_Y': 0.0,
 'kappa': 5000000.0,
 'omega': 1.0,
 'nu': nan,
 'beta_A': nan,
 'delta_A': nan,
 'alpha_O': 0.001,
 'alpha_O+': 1.0,
 'delta_O': 0.002}